In [2]:
import pandas as pd
import datetime as dt
import os
import psycopg2

In [2]:
# загрузим csv
directory = os.getcwd()

#ga_hits = pd.read_csv(f'{directory}\data\database\ga_hits.csv')
#ga_sessions = pd.read_csv(f'{directory}\data\database\ga_sessions.csv')

#для теста
ga_hits = pd.read_csv(f'{directory}\data\database\ga_hits.csv', nrows=100000)
ga_sessions = pd.read_csv(f'{directory}\data\database\ga_sessions.csv', nrows=100000)

In [9]:
ga_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   session_id                100000 non-null  object 
 1   client_id                 100000 non-null  float64
 2   visit_date                100000 non-null  object 
 3   visit_time                100000 non-null  object 
 4   visit_number              100000 non-null  int64  
 5   utm_source                99999 non-null   object 
 6   utm_medium                100000 non-null  object 
 7   utm_campaign              87836 non-null   object 
 8   utm_adcontent             81465 non-null   object 
 9   utm_keyword               41685 non-null   object 
 10  device_category           100000 non-null  object 
 11  device_os                 42325 non-null   object 
 12  device_brand              80405 non-null   object 
 13  device_model              847 non-null     ob

In [162]:
# посмотрим дубликаты
#ga_sessions[ga_sessions.session_id.duplicated()]
#ga_hits[ga_hits.duplicated()]
# не обнаружено)

,session_id,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,hit_datetime


In [7]:
# посмотрим пропуски 
# в ga_sessions самые существенные: 
        #столбец                   %пропусков    что делаем
    # device_model                99.121633      эту колонку стоит удалить
    # utm_keyword                 58.174009      удалим
    # device_os                   57.533002      постараемся заполнить
    # device_brand                19.740307      постараемся заполнить
    # utm_adcontent               18.043410      оставим как есть
    # utm_campaign                11.806346      оставим как есть

In [43]:
ga_sessions = ga_sessions.drop(columns=['device_model','utm_keyword'])

In [44]:
ga_sessions.device_os[(ga_sessions.device_os.isna()) 
                        & (ga_sessions.device_category.isin(['mobile','tablet'])) 
                        & (ga_sessions.device_brand == 'Apple')] = 'iOS'

ga_sessions.device_os[(ga_sessions.device_os.isna()) 
                        & (ga_sessions.device_category.isin(['mobile','tablet'])) 
                        & (ga_sessions.device_brand.isin(['Xiaomi','Samsung','Huawei']))] = 'Android'

ga_sessions.device_os[ga_sessions.device_os.isna()] =  '(not set)'

<ipython-input-44-41f5891a7f02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga_sessions.device_os[(ga_sessions.device_os.isna())
<ipython-input-44-41f5891a7f02>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga_sessions.device_os[(ga_sessions.device_os.isna())
<ipython-input-44-41f5891a7f02>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga_sessions.device_os[ga_sessions.device_os.isna()] =  '(not set)'


In [45]:
ga_sessions.device_brand[ga_sessions.device_brand.isna()] =  '(not set)'

<ipython-input-45-f2a7cc03c363>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga_sessions.device_brand[ga_sessions.device_brand.isna()] =  '(not set)'


In [11]:
# в ga_hits самые существенные: 
        #столбец            %пропусков    что делаем
        # event_value       100.000000    удаляем
        # hit_time           58.247795    среднее проставим
        # hit_referer        39.899634    оставляем как есть
        # event_label        23.909905    оставляем как есть

In [131]:
ga_hits = ga_hits.drop(columns=['event_value'])

In [132]:
ga_hits.hit_time[ga_hits.hit_time.isna()] = ga_hits.hit_time.mean()

<ipython-input-132-3aba3212061e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga_hits.hit_time[ga_hits.hit_time.isna()] = ga_hits.hit_time.mean()


In [14]:
#miss_values = (ga_hits.isna().sum() / len(ga_hits) * 100).sort_values(ascending = False)

In [15]:
#miss_values

In [16]:
# пребразование типов данных
# 0   session_id                object 
#  1   client_id                 object 
#  2   visit_date                object в дату
#  3   visit_time                object в тайм и объеденим
#  4   visit_number              int64 
#  5   utm_source                object
#  6   utm_medium                object
#  7   utm_campaign              object
#  8   utm_adcontent             object
#  9   device_category           object категория
#  10  device_os                 object категория
#  11  device_brand              object
#  12  device_screen_resolution  object
#  13  device_browser            object
#  14  geo_country               object
#  15  geo_city                  object
ga_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   session_id                100000 non-null  object 
 1   client_id                 100000 non-null  float64
 2   visit_date                100000 non-null  object 
 3   visit_time                100000 non-null  object 
 4   visit_number              100000 non-null  int64  
 5   utm_source                99999 non-null   object 
 6   utm_medium                100000 non-null  object 
 7   utm_campaign              87836 non-null   object 
 8   utm_adcontent             81465 non-null   object 
 9   device_category           100000 non-null  object 
 10  device_os                 100000 non-null  object 
 11  device_brand              100000 non-null  object 
 12  device_screen_resolution  100000 non-null  object 
 13  device_browser            100000 non-null  ob

In [48]:
ga_sessions['visit_datetime'] = pd.to_datetime(ga_sessions.visit_date +' '+ ga_sessions.visit_time)
ga_sessions = ga_sessions.drop(columns = ['visit_date','visit_time'], axis=1)

ga_sessions.device_category = ga_sessions.device_category.astype('category')
ga_sessions.device_os = ga_sessions.device_os.astype('category')

In [18]:
#  0   session_id      object 
#  1   hit_date        object 
#  2   hit_time        float64 объединить и сделать datetime
#  3   hit_number      int64  g
#  4   hit_type        object категория
#  5   hit_referer     object 
#  6   hit_page_path   object 
#  7   event_category  object категория
#  8   event_action    object категория
#  9   event_label     object 

In [133]:
ga_hits['hit_datetime'] = pd.to_datetime(ga_hits.hit_date) + ga_hits.hit_time.apply(lambda x: dt.timedelta(milliseconds = x))
ga_hits = ga_hits.drop(columns = ['hit_date','hit_time'], axis=1)

In [134]:
ga_hits.hit_type = ga_hits.hit_type.astype('category')
ga_hits.event_category = ga_hits.event_category.astype('category')
ga_hits.event_action = ga_hits.event_action.astype('category')

In [142]:
ga_hits.columns

Index(['session_id', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path',
       'event_category', 'event_action', 'event_label', 'hit_datetime'],
      dtype='object')

In [140]:
ga_hits

,session_id,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,hit_datetime
0,5639623078712724064.1640254056.1640254056,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,2021-12-23 00:09:57.864
1,7750352294969115059.1640271109.1640271109,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,2021-12-23 00:09:57.331
2,885342191847998240.1640235807.1640235807,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,2021-12-23 00:13:16.252
3,142526202120934167.1640211014.1640211014,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,2021-12-23 00:15:34.292
4,3450086108837475701.1640265078.1640265078,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,2021-12-23 00:12:48.741
...,...,...,...,...,...,...,...,...,...
99995,1125042169139007097.1637139064.1637139064,13,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_search_car_type_select,RRSuiTgScNuWerwiSzUU,2021-11-17 00:00:36.355
99996,4951053118655984154.1637118491.1637118491,9,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_search_car_type_select,UhNimfrZlNAYRFfFUeiH,2021-11-17 00:00:26.446
99997,4631559336254300428.1637146896.1637146896,7,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_search_car_type_select,UhNimfrZlNAYRFfFUeiH,2021-11-17 00:02:37.892
99998,4536328761566858631.1637125516.1637125516,9,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_search_car_type_select,RRSuiTgScNuWerwiSzUU,2021-11-17 00:00:47.591


In [21]:
# подключаемся к БД
conn = psycopg2.connect(database="airflow", 
                        user="airflow",
                        password="airflow", 
                        host="", #postgres
                        port=5432)

In [22]:
#тест
cursor = conn.cursor()

cursor.execute('delete from ga_sessions')
conn.commit()

test

In [24]:
print(test)

None


In [50]:
ga_sessions_test = ga_sessions

In [66]:
sql_data_sessions = tuple(map(tuple, ga_sessions_test.values))

In [67]:
cursor = conn.cursor()

cursor.executemany("INSERT INTO ga_sessions VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", sql_data_sessions)

In [76]:
conn.commit()

In [144]:
ga_hits_test = ga_hits

#ga_hits_test.hit_number = ga_hits_test.hit_number.astype('object')

In [209]:
sql_data_hits = tuple(map(tuple, ga_hits_test.values))

In [207]:
max(ga_hits_test.event_label.apply(len))

TypeError: object of type 'float' has no len()

In [179]:
sql_data_hits[45999]
#sql_data_hits

('1945459348791390137.1638690737.1638690737',
 52,
 'event',
 nan,
 'sberauto.com/cars?utm_source_initial=yandex&utm_medium_initial=cpc&utm_campaign_initial=jvsVVQVKRfBIqBDBuCkq|QYfbqJXuLdJGfOSFgZOs&utm_content_initial=yXzSAJWSuRbYeLFyMVFl|lUOhbMixPaWdNzMcDzmL|xAEYTlJDbQNwQTLGxICQ|cuvllboEpYCxUznJVVav|lFpAoUqEvHoyZfCNRuCR|zoECmSThRjpIAiOwutIs|rjbbNGNkjGThUJNtsWZQ|iJMSnuazTxCGBZrCRtti|ZIKCVYXYgRrZdwlVkEEf|HbolMJUevblAbkHClEQa|kqZPVvZmXADsCZxIQwbv|vnTlRpYmthtVDImMLSmr|OfdDOXSwHYclonQJpPxh|igNLAJodDHSgqhPwufun|HbolMJUevblAbkHClEQa&utm_term_initial=&city=1&rental_page=rental_only&rental_car=rental_only&city=18',
 'search_form',
 'search_form_search_btn',
 'ghOrjqpgskpGHSFfBBsi',
 Timestamp('2021-12-05 00:04:42.119000'))

In [104]:
sql_data

(('9055434745589932991.1637753792.1637753792',
  2108382700.1637757,
  1,
  'ZpYIoDJMcFzVoPFsHGJL',
  'banner',
  'LEoPHuyFvzoNfnzGgfcd',
  'vCIpmpaGBnIQhyYNkXqp',
  'mobile',
  'Android',
  'Huawei',
  '360x720',
  'Chrome',
  'Russia',
  'Zlatoust',
  Timestamp('2021-11-24 14:36:32')),
 ('905544597018549464.1636867290.1636867290',
  210838531.16368672,
  1,
  'MvfHsxITijuriZxsqZqt',
  'cpm',
  'FTjNLDyTrXaWYgZymFkV',
  'xhoenQgDQsgfEPYNPwKO',
  'mobile',
  'Android',
  'Samsung',
  '385x854',
  'Samsung Internet',
  'Russia',
  'Moscow',
  Timestamp('2021-11-14 08:21:30')),
 ('9055446045651783499.1640648526.1640648526',
  2108385331.164065,
  1,
  'ZpYIoDJMcFzVoPFsHGJL',
  'banner',
  'LEoPHuyFvzoNfnzGgfcd',
  'vCIpmpaGBnIQhyYNkXqp',
  'mobile',
  'Android',
  'Huawei',
  '360x720',
  'Chrome',
  'Russia',
  'Krasnoyarsk',
  Timestamp('2021-12-28 02:42:06')),
 ('9055447046360770272.1622255328.1622255328',
  2108385564.162225,
  1,
  'kjsLglQLzykiRbcDiGcD',
  'cpc',
  nan,
  'NOBKLgtu

In [210]:
cursor = conn.cursor()

cursor.executemany("INSERT INTO ga_hits VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)", sql_data_hits)

conn.commit()

In [211]:
conn.close()